Here we demonstrate how pyro can be used to do inference on complex
probabilistic models. We will examine a problem of locating signal sources in
2 dimensions. We have 2 signal sources at unknown locations, and their signals
superpose at any given point in space. For example, the signal strengths might
look like so

![signal_map](signal_map.png)



